<a href="https://colab.research.google.com/github/Farhan2898/COMP8240-FINAL_REPORT-48113832/blob/main/Report_48113832_MD_FARHAN_TANVIR_COMP8240.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296189 sha256=0f918862592f08fd06e2f88a46fb7635fb16a05b7df1b7dbbc156411868eb89b
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [2]:
!pip install nltk

# **YELPP DATASET**

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords
#nltk.download('stopwords')

# Download NLTK stopwords and wordnet
nltk.download('stopwords')
nltk.download('wordnet')

# Load the Yelp training and test datasets
yelpp_train_df = pd.read_csv('train.csv')
yelpp_test_df = pd.read_csv('test.csv')

# Adjust column names based on the structure of your dataset
yelpp_train_df.columns = ['label', 'text']
yelpp_test_df.columns = ['label', 'text']

# Get the set of stopwords for removal
stop_words = set(stopwords.words('english'))

# Precompile regex patterns for efficiency
url_pattern = re.compile(r'http\S+|www\S+|https\S+')
special_char_pattern = re.compile(r'[^a-zA-Z\s]')
extra_space_pattern = re.compile(r'\s+')

def fast_preprocess(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = url_pattern.sub('', text)

    # Remove special characters and digits
    text = special_char_pattern.sub('', text)

    # Remove extra spaces
    text = extra_space_pattern.sub(' ', text).strip()

    # Remove stop words in a vectorized way
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Join back into a string
    return ' '.join(words)

# Apply preprocessing using vectorized operations
yelpp_train_df['text'] = yelpp_train_df['text'].map(fast_preprocess)
yelpp_test_df['text'] = yelpp_test_df['text'].map(fast_preprocess)

# Prepare the training data for FastText
yelpp_train_df['fasttext_format'] = yelpp_train_df['label'].apply(lambda x: f'__label__{x}') + ' ' + yelpp_train_df['text']
preprocessed_yelpp_train_file = 'yelpp_train_fasttext.txt'
yelpp_train_df['fasttext_format'].to_csv(preprocessed_yelpp_train_file, index=False, header=False)

# Prepare the test data for FastText
yelpp_test_df['fasttext_format'] = yelpp_test_df['label'].apply(lambda x: f'__label__{x}') + ' ' + yelpp_test_df['text']
preprocessed_yelpp_test_file = 'yelpp_test_fasttext.txt'
yelpp_test_df['fasttext_format'].to_csv(preprocessed_yelpp_test_file, index=False, header=False)

print("\nOptimized preprocessing complete. Data saved for FastText.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Optimized preprocessing complete. Data saved for FastText.


## **FASTTEXT MODEL FOR YELPP WITHOUT BIGRAM Using Different Hypertuning**

### **Model build lr=0.05, epoch=10,dim=10,thread=90**

In [ ]:
import fasttext

# Define file paths for training and test data
preprocessed_yelpp_train_file = 'yelpp_train_fasttext.txt'
preprocessed_yelpp_test_file = 'yelpp_test_fasttext.txt'

# Train the FastText model
yelp_model = fasttext.train_supervised(
    input=preprocessed_yelpp_train_file,
    lr=0.05,           # Initial learning rate
    epoch=10,         # Number of epochs
    dim=10,           # Set h=10
    thread=90
)

# Save the trained model
yelp_model.save_model('fasttext_yelpp_model.bin')
print("\nModel training complete. Model saved as 'fasttext_yelpp_model.bin'.")


Model training complete. Model saved as 'fasttext_yelpp_model.bin'.


### **ACCURACY When lr=0.05, epoch=10,dim=10,thread=90. ACCURACY 93.10%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = yelp_model.test(preprocessed_yelpp_test_file)

# Print the results
print(f'\nTest Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')


Test Precision: 93.10%
Test Recall: 93.10%
Number of test examples:37999


### **Model build lr=0.05, epoch=50,dim=10,thread=90**

In [ ]:
import fasttext

# Define file paths for training and test data
preprocessed_yelpp_train_file = 'yelpp_train_fasttext.txt'
preprocessed_yelpp_test_file = 'yelpp_test_fasttext.txt'

# Train the FastText model
yelp_model = fasttext.train_supervised(
    input=preprocessed_yelpp_train_file,
    lr=0.05,           # Initial learning rate
    epoch=50,         # Number of epochs
    dim=10,           # Set h=10
    thread=90
)

# Save the trained model
yelp_model.save_model('fasttext_yelpp_model.bin')
print("\nModel training complete. Model saved as 'fasttext_yelpp_model.bin'.")


Model training complete. Model saved as 'fasttext_yelpp_model.bin'.


### **ACCURACY When lr=0.05, epoch=50,dim=10,thread=90. ACCURACY 92.49%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = yelp_model.test(preprocessed_yelpp_test_file)

# Print the results
print(f'\nTest Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')


Test Precision: 92.49%
Test Recall: 92.49%
Number of test examples:37999


## **FASTTEXT MODEL FOR YELPP WITH BIGRAM Using Different Hypertuning**

### **Model build lr=0.05,wordNgrams=2, epoch=10,dim=10,thread=90**

In [ ]:
import fasttext

# Define file paths for training and test data
preprocessed_yelpp_train_file = 'yelpp_train_fasttext.txt'
preprocessed_yelpp_test_file = 'yelpp_test_fasttext.txt'

# Train the FastText model
yelp_model = fasttext.train_supervised(
    input=preprocessed_yelpp_train_file,
    lr=0.05,           # Initial learning rate
    epoch=10,         # Number of epochs
    wordNgrams=2,     # Use bigrams
    dim=10,           # Set h=10
    thread=90
)

# Save the trained model
yelp_model.save_model('fasttext_yelpp_model.bin')
print("\nModel training complete. Model saved as 'fasttext_yelpp_model.bin'.")


Model training complete. Model saved as 'fasttext_yelpp_model.bin'.


### **PREPROCESSED YELPP DATASET**

In [ ]:
with open(preprocessed_yelpp_test_file, 'r') as file:
    for i in range(5):
        print(file.readline())

_label_1 last summer appointment get new tires wait super long time also went week fix minor problem tire put fixed free next morning issue called complain manager didnt even apologize frustrated never going back seem overpriced

_label_2 friendly staff starbucks fair get anywhere else sometimes lines get long

_label_1 food good unfortunately service hit miss main issue seems kitchen waiters waitresses often apologetic long waits pretty obvious avoid tables taking initial order avoid hearing complaints

_label_2 even didnt car filenes basement worth bus trip waterfront always find something usually find things spend better still always still wearing clothes shoes months later nni kind suspect best shopping pittsburgh much better usual department stores better marshalls tj maxx better saks downtown even sale selection bargains qualitynni like filenes better gabriel brothers harder get gabriel brothers real discount shoppers challenge im afraid didnt live pittsburgh long enough develop 

### **Accuracy with lr=0.05,wordNgrams=2, epoch=10,dim=10,thread=90  ACCURACY=94.43%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = yelp_model.test(preprocessed_yelpp_test_file)

# Print the results
print(f'\nTest Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')


Test Precision: 94.43%
Test Recall: 94.43%
Number of test examples:37999


### **Model build lr=0.05,wordNgrams=2, epoch=50,dim=10,thread=90**

In [ ]:
import fasttext

# Define file paths for training and test data
preprocessed_yelpp_train_file = 'yelpp_train_fasttext.txt'
preprocessed_yelpp_test_file = 'yelpp_test_fasttext.txt'

# Train the FastText model
yelp_model = fasttext.train_supervised(
    input=preprocessed_yelpp_train_file,
    lr=0.1,           # Initial learning rate
    epoch=50,         # Number of epochs
    wordNgrams=2,     # Use bigrams
    dim=10,           # Set h=10
    thread=90
)

# Save the trained model
yelp_model.save_model('fasttext_yelpp_model.bin')
print("\nModel training complete. Model saved as 'fasttext_yelpp_model.bin'.")


Model training complete. Model saved as 'fasttext_yelpp_model.bin'.


### **ACCURACY With lr=0.05,wordNgrams=2, epoch=50,dim=10,thread=90 ACCURACY=94.15%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = yelp_model.test(preprocessed_yelpp_test_file)

# Print the results
print(f'\nTest Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')


Test Precision: 94.15%
Test Recall: 94.15%
Number of test examples:37999


**AMAZF DATASET**

In [13]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if not already downloaded
#nltk.download('stopwords')

# Get the total number of rows in the CSV file
total_rows = sum(1 for row in open('Amaztrain.csv', 'r', encoding='utf-8'))

# Specify the number of rows to read, excluding the last 5
rows_to_read = total_rows-5

# Load the Amazon training and test datasets
amazf_train_df = pd.read_csv('Amaztrain.csv', header=None, nrows=rows_to_read)
amazf_test_df = pd.read_csv('Amaztest.csv', header=None)


# Rename the columns to 'label' and 'text'
amazf_train_df.columns = ['label', 'title', 'text']
amazf_test_df.columns = ['label', 'title', 'text']

# Select only 'label' and 'text' columns
amazf_train_df = amazf_train_df[['label', 'text']]
amazf_test_df = amazf_test_df[['label','text']]



# Get the set of stopwords for removal
stop_words = set(stopwords.words('english'))

# Precompile regex patterns for efficiency
url_pattern = re.compile(r'http\S+|www\S+|https\S+')
special_char_pattern = re.compile(r'[^a-zA-Z\s]')
extra_space_pattern = re.compile(r'\s+')

def fast_preprocess(text):
    # Check if the input is a string, otherwise convert it to an empty string
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = url_pattern.sub('', text)

    # Remove special characters and digits
    text = special_char_pattern.sub('', text)

    # Remove extra spaces
    text = extra_space_pattern.sub(' ', text).strip()

    # Remove stop words
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Join back into a string
    return ' '.join(words)

# Apply preprocessing using vectorized operations
amazf_train_df['text'] = amazf_train_df['text'].map(fast_preprocess)
amazf_test_df['text'] = amazf_test_df['text'].map(fast_preprocess)

# Prepare the training data for FastText
amazf_train_df['fasttext_format'] = amazf_train_df['label'].apply(lambda x: f'__label__{x}') + ' ' + amazf_train_df['text']
preprocessed_amazf_train_file = 'amazf_train_fasttext.txt'
amazf_train_df['fasttext_format'].to_csv(preprocessed_amazf_train_file, index=False, header=False)

# Prepare the test data for FastText
amazf_test_df['fasttext_format'] = amazf_test_df['label'].apply(lambda x: f'__label__{x}') + ' ' + amazf_test_df['text']
preprocessed_amazf_test_file = 'amazf_test_fasttext.txt'
amazf_test_df['fasttext_format'].to_csv(preprocessed_amazf_test_file, index=False, header=False)

print("\nOptimized preprocessing complete. Data saved for FastText.")



Optimized preprocessing complete. Data saved for FastText.


## **FASTTEXT MODEL FOR AMAZF WITHOUT BIGRAM Using Different Hypertuning**

### **Model build lr=0.1, epoch=10,dim=10,thread=90**

In [ ]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.1,              # Further lower learning rate for fine-tuning
    epoch=10,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    loss='softmax',       # Softmax loss for better classification
    thread=90             # Keep using multiple threads for faster training
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.1, epoch=10,dim=10,thread=90 ACCURACY=44.61%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 44.61%
Test Recall: 44.61%
Number of test examples:650000


### **Model build lr=0.1, epoch=50,dim=10,thread=90**

In [ ]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.1,              # Further lower learning rate for fine-tuning
    epoch=50,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    loss='softmax',       # Softmax loss for better classification
    thread=90             # Keep using multiple threads for faster training
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.1, epoch=50,dim=10,thread=90 ACCURACY=44.15%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 44.15%
Test Recall: 44.15%
Number of test examples:650000


### **Model build lr=0.1, epoch=20,dim=10,thread=90**

In [14]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.1,              # Further lower learning rate for fine-tuning
    epoch=20,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    loss='softmax',       # Softmax loss for better classification
    thread=90,            # Keep using multiple threads for faster training
    minn = 2,
    maxn= 6
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.1, epoch=50,dim=10,thread=90 ACCURACY=50.80%**

In [15]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 50.80%
Test Recall: 50.80%
Number of test examples:650000


## **FASTTEXT MODEL FOR AMAZF WITH BIGRAM Using Different Hypertuning**

### **Model build lr=0.1, epoch=10,dim=10,wordNgrams=2,thread=90**

In [ ]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.1,              # Further lower learning rate for fine-tuning
    epoch=10,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    wordNgrams=2,         # Capture more context using n-grams
    loss='softmax',       # Softmax loss for better classification
    thread=90             # Keep using multiple threads for faster training
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.1, epoch=50,dim=10,thread=90 ACCURACY=44.49%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 44.49%
Test Recall: 44.49%
Number of test examples:650000


### **Model build lr=0.1, epoch=20,dim=10,wordNgrams=2,thread=90**

In [16]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.1,              # Further lower learning rate for fine-tuning
    epoch=20,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    loss='softmax',       # Softmax loss for better classification
    wordNgrams=2,
    thread=90,            # Keep using multiple threads for faster training
    minn = 2,
    maxn= 6
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.1, epoch=20,dim=10,thread=90,wordNgrams=2 ACCURACY=52.91%**

In [17]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 52.91%
Test Recall: 52.91%
Number of test examples:650000


### **Model build lr=0.05, epoch=10,dim=10,wordNgrams=2,thread=90**

In [ ]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.05,              # Further lower learning rate for fine-tuning
    epoch=10,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    wordNgrams=2,         # Capture more context using n-grams
    loss='softmax',       # Softmax loss for better classification
    thread=90             # Keep using multiple threads for faster training
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.05, epoch=10,dim=10,thread=90,wordNgrams=2 ACCURACY=52.91%**

In [ ]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 44.82%
Test Recall: 44.82%
Number of test examples:650000


## Model build with Hashing Trick where lr=0.05, epoch=10,dim=10,wordNgrams=2,thread=90,minn=2,maxn=6

In [18]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.05,              # Further lower learning rate for fine-tuning
    epoch=10,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    loss='softmax',       # Softmax loss for better classification
    wordNgrams=2,
    thread=90,            # Keep using multiple threads for faster training
    minn = 2,
    maxn= 6
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.05, epoch=10,dim=10,thread=90,wordNgrams=2, minn=2, maxn=6 ACCURACY=52.52%**

In [19]:
# Evaluate the model on the test set
n_examples, precision, recall = amazf_model.test(preprocessed_amazf_test_file)

# Print results
print(f'Test Precision: {precision * 100:.2f}%')
print(f'Test Recall: {recall * 100:.2f}%')
print(f'Number of test examples:{n_examples}')

Test Precision: 52.52%
Test Recall: 52.52%
Number of test examples:650000


## Model build with Hashing Trick where lr=0.1, epoch=10,dim=10,wordNgrams=2,thread=90,minn=2,maxn=6

In [20]:
import fasttext
amazf_model = fasttext.train_supervised(
    input=preprocessed_amazf_train_file,
    lr=0.1,              # Further lower learning rate for fine-tuning
    epoch=10,            # Increase the number of epochs
    dim=10,              # Increase the dimension of embeddings
    loss='softmax',       # Softmax loss for better classification
    wordNgrams=2,
    thread=90,            # Keep using multiple threads for faster training
    minn = 2,
    maxn= 6
)

# Save the updated model
amazf_model.save_model('fasttext_amazf_model_optimized_v2.bin')

print("\nFurther optimized model training complete.")



Further optimized model training complete.


### **ACCURACY With lr=0.1, epoch=10,dim=10,thread=90,wordNgrams=2, minn=2, maxn=6 ACCURACY=52.67%**

**NEW DATASET: FAKENEWSNET DATASET**

In [ ]:
import pandas as pd
import re
import fasttext
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

# Load the dataset
df = pd.read_csv('FakeNewsNet.csv')

# Adjust column names: 'real' (0 for fake, 1 for real) and 'title' for the news headline
df = df[['real', 'title']]
df.columns = ['label', 'text']

# Clean the text: remove URLs, special characters, and lowercase everything
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and digits
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply text cleaning
df['text'] = df['text'].map(clean_text)

# Split the data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare the training data for FastText
train_df['fasttext_format'] = train_df['label'].apply(lambda x: f'__label__{x}') + ' ' + train_df['text']
preprocessed_train_file = 'fakenews_train_fasttext.txt'
train_df['fasttext_format'].to_csv(preprocessed_train_file, index=False, header=False)

# Prepare the test data for FastText
test_df['fasttext_format'] = test_df['label'].apply(lambda x: f'__label__{x}') + ' ' + test_df['text']
preprocessed_test_file = 'fakenews_test_fasttext.txt'
test_df['fasttext_format'].to_csv(preprocessed_test_file, index=False, header=False)

# Train the FastText model on the training data
fasttext_model = fasttext.train_supervised(
    input=preprocessed_train_file,
    lr=0.1,      # Learning rate
    epoch=50,    # Number of epochs
    dim=100,     # Embedding dimensions
    wordNgrams=2,  # Word n-grams
    loss='softmax', # Loss function
    thread=4     # Number of threads
)

# Save the trained model
fasttext_model.save_model('fakenews_fasttext_model.bin')

# Evaluate the model on the training set
n_train_examples, train_precision, train_recall = fasttext_model.test(preprocessed_train_file)

# Evaluate the model on the test set
n_test_examples, test_precision, test_recall = fasttext_model.test(preprocessed_test_file)

# Print the results for training and test sets
print(f'Training Set Precision (Accuracy): {train_precision * 100:.2f}%')
print(f'Training Set Recall: {train_recall * 100:.2f}%')
print(f'Number of training examples: {n_train_examples}')

print(f'Test Set Precision (Accuracy): {test_precision * 100:.2f}%')
print(f'Test Set Recall: {test_recall * 100:.2f}%')
print(f'Number of test examples: {n_test_examples}')


Training Set Precision (Accuracy): 99.49%
Training Set Recall: 99.49%
Number of training examples: 18556
Test Set Precision (Accuracy): 83.38%
Test Set Recall: 83.38%
Number of test examples: 4640


In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import re
import fasttext
import nltk  # Ensure this is imported to download resources
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize the Porter Stemmer for stemming
stemmer = PorterStemmer()

# Load the dataset
df = pd.read_csv('FakeNewsNet.csv')

# Adjust column names: 'real' (0 for fake, 1 for real) and 'title' for the news headline
df = df[['real', 'title']]
df.columns = ['label', 'text']

# Clean the text: remove URLs, special characters, convert numbers, remove short words, and apply stemming
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Remove stopwords and words with fewer than 3 characters, apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words and len(word) > 2]

    # Rejoin the words into a cleaned string
    return ' '.join(words)

# Apply text cleaning
df['text'] = df['text'].map(clean_text)

# Split the data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare the training data for FastText
train_df['fasttext_format'] = train_df['label'].apply(lambda x: f'__label__{x}') + ' ' + train_df['text']
preprocessed_train_file = 'fakenews_train_fasttext.txt'
train_df['fasttext_format'].to_csv(preprocessed_train_file, index=False, header=False)

# Prepare the test data for FastText
test_df['fasttext_format'] = test_df['label'].apply(lambda x: f'__label__{x}') + ' ' + test_df['text']
preprocessed_test_file = 'fakenews_test_fasttext.txt'
test_df['fasttext_format'].to_csv(preprocessed_test_file, index=False, header=False)

# Train the FastText model on the training data
fasttext_model = fasttext.train_supervised(
    input=preprocessed_train_file,
    lr=0.05,      # Learning rate
    epoch=25,    # Number of epochs
    dim=10,     # Embedding dimensions
    wordNgrams=2,  # Word n-grams
    loss='softmax', # Loss function
    thread=90     # Number of threads
)

# Save the trained model
fasttext_model.save_model('fakenews_fasttext_model.bin')

# Evaluate the model on the training set
n_train_examples, train_precision, train_recall = fasttext_model.test(preprocessed_train_file)

# Evaluate the model on the test set
n_test_examples, test_precision, test_recall = fasttext_model.test(preprocessed_test_file)

# Print the results for training and test sets
print(f'Training Set Precision (Accuracy): {train_precision * 100:.2f}%')
print(f'Training Set Recall: {train_recall * 100:.2f}%')
print(f'Number of training examples: {n_train_examples}')

print(f'Test Set Precision (Accuracy): {test_precision * 100:.2f}%')
print(f'Test Set Recall: {test_recall * 100:.2f}%')
print(f'Number of test examples: {n_test_examples}')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training Set Precision (Accuracy): 99.11%
Training Set Recall: 99.11%
Number of training examples: 18556
Test Set Precision (Accuracy): 83.64%
Test Set Recall: 83.64%
Number of test examples: 4640


In [ ]:
import pandas as pd
import re
import fasttext
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize the Porter Stemmer for stemming
stemmer = PorterStemmer()

# Load the dataset
df = pd.read_csv('FakeNewsNet.csv')

# Adjust column names: 'real' (0 for fake, 1 for real) and 'title' for the news headline
df = df[['real', 'title']]
df.columns = ['label', 'text']

# Clean the text: remove URLs, special characters, convert numbers, remove short words, and apply stemming
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Remove stopwords and words with fewer than 3 characters, apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words and len(word) > 2]

    # Rejoin the words into a cleaned string
    return ' '.join(words)

# Apply text cleaning
df['text'] = df['text'].map(clean_text)

# Split the data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare the training data for FastText
train_df['fasttext_format'] = train_df['label'].apply(lambda x: f'__label__{x}') + ' ' + train_df['text']
preprocessed_train_file = 'fakenews_train_fasttext.txt'
train_df['fasttext_format'].to_csv(preprocessed_train_file, index=False, header=False)

# Prepare the test data for FastText
test_df['fasttext_format'] = test_df['label'].apply(lambda x: f'__label__{x}') + ' ' + test_df['text']
preprocessed_test_file = 'fakenews_test_fasttext.txt'
test_df['fasttext_format'].to_csv(preprocessed_test_file, index=False, header=False)

# Train the FastText model with more configurations
fasttext_model = fasttext.train_supervised(
    input=preprocessed_train_file,
    lr=0.05,              # Lower learning rate for more refined learning
    epoch=100,            # More epochs to go over data multiple times
    dim=300,              # Higher dimension for richer word vectors
    wordNgrams=3,         # Capture more context with trigrams
    loss='ova',           # Switch to one-vs-all loss function
    bucket=2000000,       # Use larger bucket size to handle more n-grams
    minCount=2,           # Ignore words that appear fewer than 2 times
    thread=8              # Increase threads to speed up training
)

# Save the trained model
fasttext_model.save_model('fakenews_fasttext_model.bin')

# Evaluate the model on the training set
n_train_examples, train_precision, train_recall = fasttext_model.test(preprocessed_train_file)

# Evaluate the model on the test set
n_test_examples, test_precision, test_recall = fasttext_model.test(preprocessed_test_file)

# Print the results for training and test sets
print(f'Training Set Precision (Accuracy): {train_precision * 100:.2f}%')
print(f'Training Set Recall: {train_recall * 100:.2f}%')
print(f'Number of training examples: {n_train_examples}')

print(f'Test Set Precision (Accuracy): {test_precision * 100:.2f}%')
print(f'Test Set Recall: {test_recall * 100:.2f}%')
print(f'Number of test examples: {n_test_examples}')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training Set Precision (Accuracy): 99.49%
Training Set Recall: 99.49%
Number of training examples: 18556
Test Set Precision (Accuracy): 83.66%
Test Set Recall: 83.66%
Number of test examples: 4640


In [4]:
import pandas as pd
import re
import fasttext
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download necessary NLTK resources
#nltk.download('stopwords')
#nltk.download('punkt')

# Initialize the Porter Stemmer for stemming
stemmer = PorterStemmer()

# Load the dataset
df = pd.read_csv('FakeNewsNet.csv')

# Adjust column names: 'real' (0 for fake, 1 for real) and 'title' for the news headline
df = df[['real', 'title']]
df.columns = ['label', 'text']

# Remove empty texts or rows with NaN
df = df.dropna(subset=['text', 'label'])
df = df[df['text'].str.strip() != '']  # Remove rows with empty text fields

# Clean the text: remove URLs, special characters, convert numbers, remove short words, and apply stemming
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Remove stopwords and words with fewer than 3 characters, apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words and len(word) > 2]

    # Rejoin the words into a cleaned string
    return ' '.join(words)

# Apply text cleaning
df['text'] = df['text'].map(clean_text)

# Split the data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print ("Preprocessed Data:")
print(train_df.head())

Preprocessed Data:
       label                                               text
9198       0  chuck schumer steami affair high school cheerl...
14985      1  trace elli ross shut babi question tell peopl ...
15100      1                       friend music parodi broadway
6186       1                        jurass world fallen kingdom
11038      1  cassi join rank kim kardashian pat mcgrath new...


In [ ]:
import pandas as pd
import re
import fasttext
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize the Porter Stemmer for stemming
stemmer = PorterStemmer()

# Load the dataset
df = pd.read_csv('FakeNewsNet.csv')

# Adjust column names: 'real' (0 for fake, 1 for real) and 'title' for the news headline
df = df[['real', 'title']]
df.columns = ['label', 'text']

# Remove empty texts or rows with NaN
df = df.dropna(subset=['text', 'label'])
df = df[df['text'].str.strip() != '']  # Remove rows with empty text fields

# Clean the text: remove URLs, special characters, convert numbers, remove short words, and apply stemming
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Remove stopwords and words with fewer than 3 characters, apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words and len(word) > 2]

    # Rejoin the words into a cleaned string
    return ' '.join(words)

# Apply text cleaning
df['text'] = df['text'].map(clean_text)

# Split the data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare the training data for FastText
train_df['fasttext_format'] = train_df['label'].apply(lambda x: f'__label__{x}') + ' ' + train_df['text']
preprocessed_train_file = 'fakenews_train_fasttext.txt'
train_df['fasttext_format'].to_csv(preprocessed_train_file, index=False, header=False)

# Prepare the test data for FastText
test_df['fasttext_format'] = test_df['label'].apply(lambda x: f'__label__{x}') + ' ' + test_df['text']
preprocessed_test_file = 'fakenews_test_fasttext.txt'
test_df['fasttext_format'].to_csv(preprocessed_test_file, index=False, header=False)

# Function to evaluate the FastText model with different hyperparameters
def evaluate_model(lr, epoch, dim, wordNgrams, loss, bucket=2000000, minCount=2):
    try:
        # Train the FastText model with the provided hyperparameters
        fasttext_model = fasttext.train_supervised(
            input=preprocessed_train_file,
            lr=lr,               # Learning rate
            epoch=epoch,         # Number of epochs
            dim=dim,             # Embedding dimension size
            wordNgrams=wordNgrams,  # N-gram size
            loss=loss,           # Loss function
            bucket=bucket,       # Bucket size for n-grams
            minCount=minCount,   # Minimum word frequency threshold
            thread=8             # Number of threads for parallel training
        )

        # Evaluate on training set
        n_train_examples, train_precision, _ = fasttext_model.test(preprocessed_train_file)

        # Evaluate on test set
        n_test_examples, test_precision, _ = fasttext_model.test(preprocessed_test_file)

        # Return precision values
        return train_precision, test_precision

    except Exception as e:
        print(f"Error during model training: {e}")
        return None, None

# Hyperparameter grid (you can adjust these ranges as needed)
learning_rates = [0.1, 0.05]  # Reduced range
epochs = [10, 25, 100]
dimensions = [10]
n_grams = [1, 2, 3 ]
loss_functions = ['softmax', 'ova']

# Store results for comparison
results = []

# Loop through the hyperparameter combinations
for lr in learning_rates:
    for epoch in epochs:
        for dim in dimensions:
            for n_gram in n_grams:
                for loss in loss_functions:
                    print(f"Training model with lr={lr}, epoch={epoch}, dim={dim}, n_gram={n_gram}, loss={loss}")

                    # Evaluate the model with the current set of hyperparameters
                    train_prec, test_prec = evaluate_model(lr, epoch, dim, n_gram, loss)

                    if train_prec is not None and test_prec is not None:
                        # Store the results if valid
                        results.append({
                            'lr': lr,
                            'epoch': epoch,
                            'dim': dim,
                            'n_gram': n_gram,
                            'loss': loss,
                            'train_precision': train_prec,
                            'test_precision': test_prec
                        })

# Find the best configuration
best_result = max(results, key=lambda x: x['test_precision'])

# Print the best hyperparameters and their performance
print("\nBest hyperparameters based on test precision:")
print(f"Learning Rate: {best_result['lr']}")
print(f"Epoch: {best_result['epoch']}")
print(f"Dimension: {best_result['dim']}")
print(f"N-gram: {best_result['n_gram']}")
print(f"Loss Function: {best_result['loss']}")
print(f"Training Precision: {best_result['train_precision'] * 100:.2f}%")
print(f"Test Precision: {best_result['test_precision'] * 100:.2f}%")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training model with lr=0.1, epoch=10, dim=10, n_gram=1, loss=softmax
Error during model training: Encountered NaN.
Training model with lr=0.1, epoch=10, dim=10, n_gram=1, loss=ova
Error during model training: Encountered NaN.
Training model with lr=0.1, epoch=10, dim=10, n_gram=2, loss=softmax
Training model with lr=0.1, epoch=10, dim=10, n_gram=2, loss=ova
Training model with lr=0.1, epoch=10, dim=10, n_gram=3, loss=softmax
Training model with lr=0.1, epoch=10, dim=10, n_gram=3, loss=ova
Training model with lr=0.1, epoch=25, dim=10, n_gram=1, loss=softmax
Error during model training: Encountered NaN.
Training model with lr=0.1, epoch=25, dim=10, n_gram=1, loss=ova
Error during model training: Encountered NaN.
Training model with lr=0.1, epoch=25, dim=10, n_gram=2, loss=softmax
Training model with lr=0.1, epoch=25, dim=10, n_gram=2, loss=ova
Training model with lr=0.1, epoch=25, dim=10, n_gram=3, loss=softmax
Training model with lr=0.1, epoch=25, dim=10, n_gram=3, loss=ova
Training mod